In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format
from scipy.stats import norm as norm
from pandas_datareader.data import Options

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *

In [46]:
vol_window = 5
market_close_hour = 13

def expiration_options_price(vol_window, market_close_hour, use_iv = 'prices'):

    spx_ticker = '^GSPC'
    spx_minutely = yahoo_query(spx_ticker, dt.datetime(2018,1,1))
    spx_minutely.minute_query()
    spx_minutely = spx_minutely.minute_prices
    spx_minutely.columns = [col.replace('^', '') for col in spx_minutely.columns.tolist()]
    spx_minutely['returns'] = np.log(spx_minutely.GSPC_close) - np.log(spx_minutely.GSPC_close.shift(1))

    vix_ticker = '^VIX'
    vix_minutely = yahoo_query(vix_ticker, dt.datetime(2018,1,1))
    vix_minutely.minute_query()
    vix_minutely = vix_minutely.minute_prices
    vix_minutely.columns = [col.replace('^', '') for col in vix_minutely.columns.tolist()]

    current_minute_vol = spx_minutely.returns.rolling(vol_window).std().tail(1).values[0]
    latest_vix = vix_minutely.dropna().tail(1).VIX_close.values[0]
    latest_stdev = latest_vix/(100*np.sqrt(252))

    latest_spx = spx_minutely.dropna().tail(1).GSPC_close.values[0]
    latest_range = [latest_spx*(1 - 2*latest_stdev), latest_spx*(1 + 2*latest_stdev)]
    strike_range = list(range(int(np.floor(latest_range[0]/10)*10), int(np.ceil(latest_range[1]/10)*10) + 5,5))

    current_time = dt.datetime.now()
    market_close = dt.datetime(current_time.year, current_time.month, current_time.day, market_close_hour, 0)

    minutes_to_close = (market_close - current_time).seconds/60

    raw_chain = pd.DataFrame({'Strike': strike_range})
    raw_chain['Underlying_Price'] = latest_spx
    
    if type(use_iv) == str:
        raw_chain['IV'] = current_minute_vol
    else:
        raw_chain['IV'] = use_iv
    raw_chain['MTE'] = minutes_to_close

    options_df = raw_chain.copy()
    options_df['sigma_sqrt_T'] = np.sqrt(options_df.MTE)*options_df.IV
    options_df['d1'] = (np.log(options_df.Underlying_Price/options_df.Strike) + ((options_df.IV**2)/2)*options_df.MTE)/(options_df.IV*np.sqrt(options_df.MTE))
    options_df['d2'] = options_df.d1 - options_df.sigma_sqrt_T
    options_df['Nd1'] = norm.cdf(options_df.d1)
    options_df['Nd2_PV'] = norm.cdf(options_df.d2)*options_df.Strike

    options_df['Call_Price'] = np.round(options_df.Nd1*options_df.Underlying_Price - options_df.Nd2_PV, 2)

    options_df['Put_Price'] = np.round(options_df.Call_Price + options_df.Strike - options_df.Underlying_Price, 2)

    options_df = options_df[['Strike','Underlying_Price','IV','MTE','Call_Price','Put_Price']].set_index('Strike')
    return options_df, spx_minutely, vix_minutely

In [83]:
vol_window = 60
market_close_hour = 13
use_iv = 0.0005
minutes = 97740
options_df, spx_minutely, vix_minutely = expiration_options_price(vol_window, market_close_hour, use_iv)

downs = spx_minutely[spx_minutely.returns < 0][['returns']]
ups = spx_minutely[spx_minutely.returns > 0][['returns']]

spx_bands = spx_minutely[['GSPC_close']].join(vix_minutely[['VIX_close']]).dropna()


current_time = spx_bands.index[-1] - dt.timedelta(seconds = 60*60*4)
market_close = dt.datetime(current_time.year, current_time.month, current_time.day, market_close_hour, 0)

minutes_to_close = (market_close - current_time).seconds/60
days_to_close = minutes_to_close/390

spx_bands['UB'] = (1 + spx_bands['VIX_close']/(100*np.sqrt(252/days_to_close)))*spx_bands.GSPC_close
spx_bands['LB'] = (1 - spx_bands['VIX_close']/(100*np.sqrt(252/days_to_close)))*spx_bands.GSPC_close

spx_bands

,GSPC_close,VIX_close,UB,LB
2019-01-11 10:31:00,"2,582.7000",20.0600,"2,585.0371","2,580.3628"
2019-01-11 10:32:00,"2,582.9500",20.0700,"2,585.2885","2,580.6114"
2019-01-11 10:33:00,"2,583.5100",20.0500,"2,585.8467","2,581.1733"
2019-01-11 10:34:00,"2,584.5601",20.0200,"2,586.8942","2,582.2259"
2019-01-11 10:35:00,"2,583.6201",20.0000,"2,585.9511","2,581.2891"
2019-01-11 10:36:00,"2,583.1399",19.9600,"2,585.4658","2,580.8140"
2019-01-11 10:37:00,"2,582.2600",20.0200,"2,584.5921","2,579.9279"
2019-01-11 10:38:00,"2,583.0601",19.9600,"2,585.3859","2,580.7342"
2019-01-11 10:39:00,"2,582.1799",20.0400,"2,584.5143","2,579.8456"
2019-01-11 10:41:00,"2,582.4500",19.9900,"2,584.7787","2,580.1212"
